In [1]:
import csv
import datetime
import os
import threading
import time
from flask import Flask, render_template, request, redirect, url_for
import cv2
import math
import pygame
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
from ultralytics import YOLO

app = Flask(__name__)

# Initialize pygame mixer
pygame.mixer.init()

# Load the alarm sound
alarm_sound = pygame.mixer.Sound("alarm.wav")

# Load YOLO model
model = YOLO("yolov8n.pt")
classNames = {
15: 'cat', 16: 'dog', 17: 'Horse', 18: 'sheep',
19: 'cow', 20: 'elephant', 21: 'bear', 22: "zebra", 23: "giraffe"
}

# Define wild and non-wild animals
wild_animals = {'Horse', 'elephant', 'bear', 'zebra', 'giraffe'}
non_wild_animals = {'cat', 'dog', 'sheep', 'cow'}

# Email configuration
sender_email = "swethanairscipytechnologies@gmail.com"
receiver_email = "viveksura99@gmail.com"
password = "kqxc gxyo nqqx sskb"

# Path to the CSV file
csv_file_path = 'animal_detection_log.csv'

# Create or open the CSV file and write header if it does not exist
def initialize_csv():
if not os.path.isfile(csv_file_path):
with open(csv_file_path, mode='w', newline='') as file:
writer = csv.writer(file)
writer.writerow(['Animal Name', 'Date and Time'])

initialize_csv()

def play_alarm():
"""Play the alarm sound in a loop."""
alarm_sound.play(-1) # Play in a loop

def stop_alarm():
"""Stop the alarm sound."""
pygame.mixer.stop()

def log_detection(animal_name):
"""Log the detected animal and current timestamp to the CSV file."""
with open(csv_file_path, mode='a', newline='') as file:
writer = csv.writer(file)
writer.writerow([animal_name, datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')])

@app.route('/', methods=['POST', 'GET'])
def index():
if request.method == 'POST':
file = request.files['file']
file_type = request.form['fileType']
file_path = os.path.join('uploads', file.filename)
file.save(file_path)
print("Video saved at:", file_path)
output_path = os.path.join('static', 'processed_' + os.path.splitext(file.filename)[0] + '.mp4')

if file_type == 'image':
process_image(file_path, output_path)
elif file_type == 'video':
process_video(file_path, output_path)

return redirect(url_for('result'))
return render_template('index.html')

def process_image(image_path, output_path):
img = cv2.imread(image_path)
detected_animals = set()
results = model(img, stream=True, verbose=False)

wild_animal_detected = False # Flag to track wild animal detection

for r in results:
boxes = r.boxes
for box in boxes:
confidence = math.ceil((box.conf[0] * 100)) / 100
if confidence > 0.8:
cls = int(box.cls[0])
if cls in classNames:
animal_name = classNames[cls]
x1, y1, x2, y2 = box.xyxy[0]
x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
org = [x1, y1 - 10]
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
color = (0, 255, 0)
thickness = 1
cv2.putText(img, animal_name, tuple(org), font, fontScale, color, thickness)

if animal_name not in detected_animals:
detected_animals.add(animal_name)
detected_image_path = os.path.join('static', f'detected_{animal_name}.jpg')
cv2.imwrite(detected_image_path, img)
send_email_alert(detected_image_path, animal_name)
log_detection(animal_name)

if animal_name in wild_animals and not wild_animal_detected:
play_alarm()
wild_animal_detected = True

# Ensure the alarm is stopped after processing completes, even for non-wild animals
stop_alarm()
cv2.imwrite(output_path, img)

def process_video(video_path, output_path):
print("Processing video...")
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'H264')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

detected_animals = set()
wild_animal_detected = False # Flag to track wild animal detection

while True:
ret, frame = cap.read()
if not ret:
break

results = model(frame, stream=True, verbose=False)
for r in results:
boxes = r.boxes
for box in boxes:
confidence = math.ceil((box.conf[0] * 100)) / 100
if confidence > 0.8:
cls = int(box.cls[0])
if cls in classNames:
animal_name = classNames[cls]
x1, y1, x2, y2 = box.xyxy[0]
x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
frame = cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
org = [x1, y1 - 10]
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
color = (0, 255, 0)
thickness = 1
frame = cv2.putText(frame, animal_name, tuple(org), font, fontScale, color, thickness)

if animal_name not in detected_animals:
detected_animals.add(animal_name)
detected_image_path = os.path.join('static', f'detected_{animal_name}.jpg')
cv2.imwrite(detected_image_path, frame)
send_email_alert(detected_image_path, animal_name)
log_detection(animal_name)

if animal_name in wild_animals and not wild_animal_detected:
play_alarm()
wild_animal_detected = True

out.write(frame)

# Ensure the alarm is stopped after processing completes, even for non-wild animals
stop_alarm()
print("Processed video saved at:", output_path)
cap.release()
out.release()

def send_email_alert(image_path, animal_name):
msg = MIMEMultipart()
msg['From'] = sender_email
msg['To'] = receiver_email
if animal_name in wild_animals:
msg['Subject'] = "Wild Animal Detection Alert"
body = f"A wild animal ({animal_name}) was detected. Please find the attached picture."
else:
msg['Subject'] = "Non-Wild Animal Detection"
body = f"A non-wild animal ({animal_name}) was detected. Please find the attached picture."
msg.attach(MIMEText(body, 'plain'))

with open(image_path, "rb") as attachment:
part = MIMEBase("application", "octet-stream")
part.set_payload(attachment.read())
encoders.encode_base64(part)
part.add_header(
"Content-Disposition",
f"attachment; filename= {os.path.basename(image_path)}",
)
msg.attach(part)

try:
server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()
server.login(sender_email, password)
text = msg.as_string()
server.sendmail(sender_email, receiver_email, text)
server.quit()
print(f"Email sent to {receiver_email} with picture.")
except Exception as e:
print(f"Failed to send email: {e}")

@app.route('/result')
def result():
files = os.listdir('static')
video_files = [f for f in files if f.endswith('.mp4')]
print("Video files in 'static' folder:", video_files) # Debug statement
if video_files:
latest_video = max(video_files, key=lambda x: os.path.getmtime(os.path.join('static', x)))
return render_template('result.html', filename=latest_video)
else:
return "No processed video available."

if __name__ == '__main__':
app.run(debug=True)


IndentationError: expected an indented block after function definition on line 45 (ipython-input-1495622601.py, line 46)